In [30]:
import pyvista as pv
from dataclasses import dataclass
import pandas as pd
import numpy as np

In [31]:
@dataclass
class planet():
    name:str
    initial_position:np.array
    initial_velocity:np.array
    mass:int
    

In [32]:
terra = planet(
    name="Terra",
    initial_position=np.array([0,0,0]),
    initial_velocity=np.array([0,0,0]),
    mass=6e24,
)

lua = planet(
    name="Lua",
    initial_position=np.array([380e6,0,0]),
    initial_velocity=np.array([0,0.7*1026.234814619365,0]),
    mass=7e22,
)

planets_list = [terra,lua]

In [33]:
tf = 14*24 #horas
time_step = 6*60 #secs
time_vector = np.arange(0,tf*60*60+time_step,time_step)


def compute_aceleration_vector(body_position,body_mass,list_other_bodies_positions,list_other_bodies_masses):
    G = 6.67*10**(-11) # Nm2/kg2

    force_vector = np.zeros(3)

    for other_body_position,other_body_mass in zip(list_other_bodies_positions,list_other_bodies_masses):
        vector_r = other_body_position-body_position
        force_vector += G*body_mass*other_body_mass/np.abs(np.sum(vector_r**3))*vector_r

    aceleration = force_vector/body_mass
    
    return aceleration

dict_planets_history = {}
for planet in planets_list:
    df = pd.DataFrame(columns=["time","position","velocity"])
    df = df.set_index("time")
    df.loc[time_vector[0]] = {
        "position":planet.initial_position,
        "velocity":planet.initial_velocity,
    }
    dict_planets_history.update({planet.name:df})

for time in time_vector:
    for planet in planets_list:
        aceleration = compute_aceleration_vector(
            body_position=dict_planets_history[planet.name].loc[time,"position"],
            body_mass=planet.mass,
            list_other_bodies_positions = [
                other_planet_df.loc[time,"position"] 
                for other_planet_name,other_planet_df 
                in dict_planets_history.items() 
                if other_planet_name != planet.name
            ],
            list_other_bodies_masses = [
               other_planet.mass for other_planet in planets_list if other_planet != planet
            ],
        )

        velocity = dict_planets_history[planet.name].loc[time,"velocity"]
        position = dict_planets_history[planet.name].loc[time,"position"]

        new_velocity = velocity + time_step * aceleration
        new_position = position + time_step * (new_velocity+velocity)/2

        dict_planets_history[planet.name].loc[time+time_step] = {
            "position":new_position,
            "velocity":new_velocity,
        }


In [34]:
filename = "two_body.mp4"

raio_terra = 5*6370e3 #m
raio_lua = 5*1735e3 #m

mesh_terra_initial = pv.Sphere(radius=raio_terra)
mesh_terra = pv.Sphere(radius=raio_terra)
mesh_lua_initial = pv.Sphere(radius=raio_lua)
mesh_lua = pv.Sphere(radius=raio_lua)

In [35]:
cena = pv.Plotter()
cena.open_movie(filename)
cena.add_mesh(mesh_terra,scalars=None)
cena.add_mesh(mesh_lua,scalars=None)
# cena.show(auto_close=False)  
cena.camera.focal_point = (0, 0, 0)
cena.camera.position = (1.5*380e6, 1.5*380e6, 1.5*380e6)
cena.camera.zoom(0.8)
cena.camera.clipping_range = (1.5*380e6/2, 1.5*2*380e6)

number_of_frames = 101
for simulation_slice in np.linspace(0,1,number_of_frames):

    iloc = int(simulation_slice * (len(time_vector)))
    mesh_terra.points = mesh_terra_initial.points + dict_planets_history["Terra"].iloc[iloc]["position"]
    mesh_lua.points = mesh_lua_initial.points + dict_planets_history["Lua"].iloc[iloc]["position"]
    # cena.reset_camera()
    cena.add_text(f"Time: {dict_planets_history["Terra"].iloc[iloc].name//3600}h", name='time-label')
    cena.write_frame()  # Write this frame

cena.close()